In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('train_data.csv')

In [2]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,trafficSource.isTrueDirect,purchaseValue,browser,device.screenResolution,trafficSource.adContent,trafficSource.keyword,screenSize,geoCluster,trafficSource.adwordsClickInfo.slot,device.mobileDeviceBranding,...,device.language,deviceType,userChannel,device.browserVersion,totalHits,device.screenColors,sessionStart,geoNetwork.continent,device.isMobile,new_visits
0,NaN,0.0,Edge,not available in demo dataset,NaN,NaN,medium,Region_2,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Social,not available in demo dataset,1,not available in demo dataset,1500100799,Americas,False,1.0
1,True,0.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Direct,not available in demo dataset,1,not available in demo dataset,1495262065,Americas,False,1.0
2,True,0.0,Chrome,not available in demo dataset,NaN,(not provided),medium,Region_2,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Organic Search,not available in demo dataset,6,not available in demo dataset,1508510328,Europe,False,NaN
3,NaN,0.0,Internet Explorer,not available in demo dataset,NaN,NaN,medium,Region_4,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Social,not available in demo dataset,1,not available in demo dataset,1483431838,Asia,False,1.0
4,True,88950000.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,...,not available in demo dataset,desktop,Direct,not available in demo dataset,66,not available in demo dataset,1475804633,Americas,False,1.0


In [3]:
X = df.drop(['purchaseValue'],axis=1)
Y = df['purchaseValue']

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116023 entries, 0 to 116022
Data columns (total 51 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   trafficSource.isTrueDirect                    42890 non-null   object 
 1   browser                                       116023 non-null  object 
 2   device.screenResolution                       116023 non-null  object 
 3   trafficSource.adContent                       2963 non-null    object 
 4   trafficSource.keyword                         44162 non-null   object 
 5   screenSize                                    116023 non-null  object 
 6   geoCluster                                    116023 non-null  object 
 7   trafficSource.adwordsClickInfo.slot           4281 non-null    object 
 8   device.mobileDeviceBranding                   116023 non-null  object 
 9   device.mobileInputSelector                    11

In [5]:
X_cat = ['browser','geoCluster','trafficSource.adwordsClickInfo.slot','geoNetwork.networkDomain','os','geoNetwork.subContinent'
        ,'trafficSource.medium','trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.page','deviceType','userChannel'
        ,'geoNetwork.continent','geoNetwork.region','trafficSource','locationCountry','geoNetwork.city','geoNetwork.metro',
         'trafficSource.referralPath','trafficSource.isTrueDirect','trafficSource.adwordsClickInfo.isVideoAd','device.isMobile']
X_num = list(X.select_dtypes(include=['int64','float64']).columns)

# print(sorted(list(X.columns)),sorted(list(X_cat)+list(X_num)))

# assert(sorted(list(X.columns))==sorted(list(X_cat)+list(X_num)))

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

num_t = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

cat_t = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(sparse_output=False))
])

ct = ColumnTransformer(transformers=[
    ('num_t',num_t,X_num),
    ('cat_t',cat_t,X_cat)
])

X_new = ct.fit_transform(X)

In [7]:
from sklearn.model_selection import GridSearchCV,train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_new,Y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'max_depth': [20, 30],
    'min_samples_split': [2, 5],
    'n_estimators': [50, 100]
}

gc = GridSearchCV(RandomForestRegressor(random_state=42),param_grid,cv=3,scoring='r2',n_jobs=-1)
gc.fit(X_train,Y_train)

In [ ]:
gc.best_params_

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(Y_test,gc.best_estimator_.predict(X_test)))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

param_grid = {
    'n_estimators': [10,20,30],
    'learning_rate': [3,9]
}

gc = GridSearchCV(AdaBoostRegressor(random_state=42),param_grid,cv=3,scoring='r2',n_jobs=-1)
gc.fit(X_train,Y_train)

In [ ]:
gc.best_params_

In [ ]:
print(r2_score(Y_test,gc.best_estimator_.predict(X_test)))